In [78]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sakhawat18/asteroid-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/benjaminbrooke/.cache/kagglehub/datasets/sakhawat18/asteroid-dataset/versions/275


In [79]:
with open("/Users/benjaminbrooke/.cache/kagglehub/datasets/sakhawat18/asteroid-dataset/versions/275/dataset.csv") as f:
    read_file = f.read()

In [80]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number
spark = SparkSession.builder.appName("ExtractNestedData").getOrCreate()

In [81]:
df = spark.read.format("csv").option('header', 'true').load("/Users/benjaminbrooke/.cache/kagglehub/datasets/sakhawat18/asteroid-dataset/versions/275/dataset.csv")

In [82]:
print(df.columns)
print(df.count())
print(len(df.columns))

['id', 'spkid', 'full_name', 'pdes', 'name', 'prefix', 'neo', 'pha', 'H', 'diameter', 'albedo', 'diameter_sigma', 'orbit_id', 'epoch', 'epoch_mjd', 'epoch_cal', 'equinox', 'e', 'a', 'q', 'i', 'om', 'w', 'ma', 'ad', 'n', 'tp', 'tp_cal', 'per', 'per_y', 'moid', 'moid_ld', 'sigma_e', 'sigma_a', 'sigma_q', 'sigma_i', 'sigma_om', 'sigma_w', 'sigma_ma', 'sigma_ad', 'sigma_n', 'sigma_tp', 'sigma_per', 'class', 'rms']
958524
45


In [83]:
from pyspark.sql.window import Window

df = df.withColumn("Index", row_number().over(Window.partitionBy().orderBy("id")))

df = df.filter(col("Index") < 10000)

In [84]:
df.select([x for x,i in zip(df.columns,range(17))]).show(50)

25/11/17 21:56:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+-------+-----------------+----+----------+------+---+---+----+--------+------+--------------+--------+---------+---------+----------------+-------+
|      id|  spkid|        full_name|pdes|      name|prefix|neo|pha|   H|diameter|albedo|diameter_sigma|orbit_id|    epoch|epoch_mjd|       epoch_cal|equinox|
+--------+-------+-----------------+----+----------+------+---+---+----+--------+------+--------------+--------+---------+---------+----------------+-------+
|a0000001|2000001|          1 Ceres|   1|     Ceres|  NULL|  N|  N| 3.4|   939.4| 0.090|           0.2|  JPL 47|2458600.5|    58600|20190427.0000000|  J2000|
|a0000002|2000002|         2 Pallas|   2|    Pallas|  NULL|  N|  N| 4.2|     545| 0.101|            18|  JPL 37|2459000.5|    59000|20200531.0000000|  J2000|
|a0000003|2000003|           3 Juno|   3|      Juno|  NULL|  N|  N|5.33| 246.596| 0.214|        10.594| JPL 112|2459000.5|    59000|20200531.0000000|  J2000|
|a0000004|2000004|          4 Vesta|   4|     Vesta|

In [85]:
df_features = df.select([x for x,i in zip(df.columns,range(27)) if i >= 17 and i < 26])

(df_features.columns)
df_features.show()

25/11/17 21:56:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|                 e|                a|                q|                i|               om|                w|               ma|               ad|                n|
+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| .0760090265983052|2.769165148633284|2.558683601195717|10.59406719506626|80.30553090445737|73.59769469844186|77.37209751948711|2.979646696070851|.2138852265918273|
| .2299722588646258|2.773841434873298|2.135934854363191|34.83293159121413|173.0247412488342|310.2023924446679|144.9756754788195|3.411748015383405| .213344586343708|
| .2569364345017789| 2.66828528813587|1.982705579968687|12.99104349619575|169.8514824901798|248.0661932666256|125.4353546073513|3.353864996303052|.2261286975791392|
|.08872145

In [87]:
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import col

df_features = df.select([x for x in df.columns[17:40]])

for column_header in df_features.columns:
    df_features = df_features.withColumn(column_header,col(column_header).try_cast(FloatType()))

df_features.schema

StructType([StructField('e', FloatType(), True), StructField('a', FloatType(), True), StructField('q', FloatType(), True), StructField('i', FloatType(), True), StructField('om', FloatType(), True), StructField('w', FloatType(), True), StructField('ma', FloatType(), True), StructField('ad', FloatType(), True), StructField('n', FloatType(), True), StructField('tp', FloatType(), True), StructField('tp_cal', FloatType(), True), StructField('per', FloatType(), True), StructField('per_y', FloatType(), True), StructField('moid', FloatType(), True), StructField('moid_ld', FloatType(), True), StructField('sigma_e', FloatType(), True), StructField('sigma_a', FloatType(), True), StructField('sigma_q', FloatType(), True), StructField('sigma_i', FloatType(), True), StructField('sigma_om', FloatType(), True), StructField('sigma_w', FloatType(), True), StructField('sigma_ma', FloatType(), True), StructField('sigma_ad', FloatType(), True)])

Basic Column Definition
SPK-ID: Object primary SPK-ID
Object ID: Object internal database ID
Object fullname: Object full name/designation
pdes: Object primary designation
name: Object IAU name
NEO: Near-Earth Object (NEO) flag
PHA: Potentially Hazardous Asteroid (PHA) flag
H: Absolute magnitude parameter
Diameter: object diameter (from equivalent sphere) km Unit
Albedo: Geometric albedo
Diameter_sigma: 1-sigma uncertainty in object diameter km Unit
Orbit_id: Orbit solution ID
Epoch: Epoch of osculation in modified Julian day form
Equinox: Equinox of reference frame
e: Eccentricity
a: Semi-major axis au Unit
q: perihelion distance au Unit
i: inclination; angle with respect to x-y ecliptic plane
tp: Time of perihelion passage TDB Unit
moid_ld: Earth Minimum Orbit Intersection Distance au Unit

In [88]:
import torch
from torch import nn
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="neo",outputCol="neo_hot_coded")
df = indexer.fit(df).transform(df)

df_neo = df.select(col("neo_hot_coded"))

array_neo = df_neo.rdd.flatMap(lambda x: x).collect()

y = torch.tensor(array_neo)

df_features = df_features.rdd.map(lambda row: [x for x in row]).collect()

X = torch.tensor(df_features)

25/11/17 21:56:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 21:56:27 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/17 2

In [89]:
print(len(df_features),len(df_features[0]))
print(df_features[0])

9999 23
[0.07600902765989304, 2.7691650390625, 2.5586836338043213, 10.594067573547363, 80.30553436279297, 73.59769439697266, 77.37210083007812, 2.979646682739258, 0.21388523280620575, 2458238.75, 20180430.0, 1683.145751953125, 4.60820198059082, 1.5947799682617188, 620.6405029296875, 4.818999799899304e-12, 1.032799974015619e-11, 1.9568999198860126e-11, 4.608899928371102e-09, 6.168799870920338e-08, 6.624799908649948e-08, 7.820699643446005e-09, 1.1113000276952167e-11]


In [90]:
from torch.utils.data import DataLoader, TensorDataset

X_train, y_train =  X[:8000], y[:8000]
X_test, y_test =  X[8000:], y[8000:]


train_dataset = TensorDataset(X_train,y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset,batch_size=50,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=50,shuffle=True)

In [91]:
class AstroidClassificationLinear(nn.Module):
    def __init__(self):
        super().__init__()

        self.Linear_one =  nn.Linear(in_features=23,out_features=200)
        self.Linear_two =  nn.Linear(in_features=200,out_features=300)
        self.Linear_three =  nn.Linear(in_features=300,out_features=200)
        self.Linear_four =  nn.Linear(in_features=200,out_features=100)
        self.Linear_five =  nn.Linear(in_features=100,out_features=1)

    def forward(self,x):
        x = self.Linear_two(self.Linear_one(x))
        x = self.Linear_four(self.Linear_three(x))
        return self.Linear_five(x)

In [92]:
model_liner_classify = AstroidClassificationLinear()

In [93]:
optimizer = torch.optim.Adam(model_liner_classify.parameters(),lr=0.01)

In [94]:
loss_fn = nn.BCELoss()

In [ ]:
epochs = 3

for epoch in range(epochs):




